Librerias

In [90]:
import math
import random
import re
import pandas as pd

Datos

In [91]:
archivo = r"C:\Users\afpue\Documents\GitHub\NLP-Seminar\publicaciones\binaria_limpio.xlsx"
df = pd.read_excel(archivo)
df['Clase'] = df['Clase'].map({'Negativo': 0, 'Positivo': 1})

df['Documento'] = df['Documento_Lematizado']
def limpiar(texto):
    texto = texto.lower()
    return re.findall(r'\b\w+\b', texto)

Diccionarios

In [92]:
palabras_positivas = {"feliz", "alegre", "contento", "maravilloso", "excelente", "genial", "bueno", "positivo", "fantástico", "me encanta",
                      "filantropos", "filantropo"}
palabras_negativas = {"triste", "deprimido", "mal", "horrible", "terrible", "enfermo", "negativo", "odio", "me molesta", "estresado", 
                      "muertes", "muerte", "enfermedad", "dolor", "sufrimiento", "tragedia", "desastre", "crisis", "problema", "conflicto",
                      "infectados", "desgracia"}


Funciones

In [93]:
def extraer_caracteristicas(tweet, palabras_negativas, palabras_positivas):
    tokens = limpiar(tweet)
    x0 = 1  # Bias
    x1 = sum(1 for palabra in tokens if palabra in palabras_negativas)
    x2 = sum(1 for palabra in tokens if palabra in palabras_positivas)
    x3 = len(tokens)
    return [x0, x1, x2, x3]

def sigmoid(z):
    return 1 / (1 + math.exp(-z))

def dot(w, x):
    return sum(wi * xi for wi, xi in zip(w, x))

def predict_proba(w, x):
    return sigmoid(dot(w, x))

def single_log_loss(p, y):
    p = max(min(p, 1 - 1e-15), 1e-15)
    return - y * math.log(p) - (1 - y) * math.log(1 - p)

def compute_batch_gradient(w, X_batch, y_batch):
    grad = [0.0 for _ in w]
    batch_size = len(y_batch)
    for xi, yi in zip(X_batch, y_batch):
        p = predict_proba(w, xi)
        for j in range(len(w)):
            grad[j] += (p - yi) * xi[j]
    return [g / batch_size for g in grad]

def train_logistic_regression_minibatch(X, y, lr=0.01, epochs=1000, batch_size=10):
    w = [0.0 for _ in range(len(X[0]))]  # Inicializar pesos

    for epoch in range(epochs):
        combined = list(zip(X, y))
        random.shuffle(combined)

        # Dividir en mini-batches
        for i in range(0, len(y), batch_size):
            batch = combined[i:i + batch_size]
            X_batch, y_batch = zip(*batch)

            # Calcular gradiente promedio en el batch
            grad = compute_batch_gradient(w, X_batch, y_batch)

            # Actualizar pesos
            w = [wj - lr * gj for wj, gj in zip(w, grad)]

        # (Opcional) Calcular pérdida promedio en la época
        loss_epoch = sum(single_log_loss(predict_proba(w, xi), yi) for xi, yi in zip(X, y)) / len(y)
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Época {epoch}: pérdida promedio = {loss_epoch:.4f}")

    return w

def predict(w, x, threshold=0.5):
    return 1 if predict_proba(w, x) >= threshold else 0


In [94]:
tweet_test = df.iloc[-1]['Documento']
y_test = df.iloc[-1]['Clase']
x_test = extraer_caracteristicas(tweet_test, palabras_negativas, palabras_positivas)

# Resto como entrenamiento
df_train = df.iloc[:-1]
X_train = [extraer_caracteristicas(texto, palabras_negativas, palabras_positivas) for texto in df_train['Documento']]
y_train = df_train['Clase'].tolist()


In [95]:
w = train_logistic_regression_minibatch(X_train, y_train, lr=0.01, epochs=1000, batch_size=5)

# Predicción final sobre un ejemplo de prueba
proba = predict_proba(w, x_test)
clase_predicha = predict(w, x_test)

print("\n--- Resultado final ---")
print("Tweet:", tweet_test)
print("Clase real:", y_test)
print("Clase predicha:", clase_predicha)
print(f"Probabilidad estimada de clase positiva: {proba:.4f}")


Época 0: pérdida promedio = 0.7275
Época 100: pérdida promedio = 0.6592
Época 200: pérdida promedio = 0.6535
Época 300: pérdida promedio = 0.6386
Época 400: pérdida promedio = 0.6116
Época 500: pérdida promedio = 0.5830
Época 600: pérdida promedio = 0.5814
Época 700: pérdida promedio = 0.5579
Época 800: pérdida promedio = 0.5489
Época 900: pérdida promedio = 0.5581
Época 999: pérdida promedio = 0.5322

--- Resultado final ---
Tweet: el mundo yo parecer más amable , más humano , menos raro .
Clase real: 1
Clase predicha: 1
Probabilidad estimada de clase positiva: 0.6853


In [96]:
w

[0.6468778796656445, -1.6846246375238016, 0.0, 0.013156821027996254]